In [1]:
#!/usr/bin/env python3
"""
Autoencoder for IoT Intrusion Detection - PyTorch Implementation
Dense Layer Architecture
60-20-20 Train-Val-Test Split
Max 5 files loaded at once
GPU Accelerated
"""

import os
import sys
import gc
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import kagglehub
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
import warnings
warnings.filterwarnings('ignore')

# ==========================================================
# 🎮 GPU CONFIGURATION
# ==========================================================

def setup_gpu():
    """Configure PyTorch to use GPU efficiently"""
    print("=" * 80)
    print("🎮 GPU Configuration")
    print("=" * 80)

    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"✅ CUDA Version: {torch.version.cuda}")
        print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        print(f"✅ Number of GPUs: {torch.cuda.device_count()}")
        torch.backends.cudnn.benchmark = True
        print("✅ cuDNN autotuner enabled")
    else:
        device = torch.device('cpu')
        print("⚠️  No GPU detected, running on CPU")

    print("=" * 80 + "\n")
    return device

device = setup_gpu()

# ==========================================================
# 🧹 HELPER FUNCTIONS
# ==========================================================

def load_and_clean(path, label_col=None):
    """Load CSV and separate features from labels"""
    df = pd.read_csv(path)
    df = df.dropna()
    df = df.drop_duplicates()

    if label_col is None:
        label_col = "Label" if "Label" in df.columns else df.columns[-1]

    X = df.drop(columns=[label_col])
    y = df[label_col]
    return X, y


def encode_objects(X):
    """Encode categorical columns and convert to numpy array"""
    for col in X.select_dtypes(include=["object"]).columns:
        X[col] = LabelEncoder().fit_transform(X[col])
    return X.values


def process_files_generator(file_list, scaler, pca, label_encoder, batch_size=5):
    """Generator that yields batches of processed data without storing all in memory"""
    for i in range(0, len(file_list), batch_size):
        batch_files = file_list[i:i+batch_size]

        X_batch = []
        y_batch = []

        for f in batch_files:
            try:
                X, y = load_and_clean(f)
                X = encode_objects(X)

                X_scaled = scaler.transform(X)
                X_reduced = pca.transform(X_scaled)

                X_batch.append(X_reduced)
                y_batch.append(label_encoder.transform(y.astype(str)))

            except Exception as e:
                print(f"Error processing {f}: {e}")
                continue

        if X_batch:
            X_combined = np.vstack(X_batch)
            y_combined = np.hstack(y_batch)

            del X_batch, y_batch
            gc.collect()

            yield X_combined, y_combined


# ==========================================================
# 🏗️ AUTOENCODER MODEL
# ==========================================================

class DenseAutoencoder(nn.Module):
    """
    Autoencoder with Dense (Fully Connected) Layers
    Encoder: input -> 128 -> 64 -> 32 (bottleneck)
    Decoder: 32 -> 64 -> 128 -> output
    Classifier: 32 (bottleneck) -> num_classes
    """

    def __init__(self, input_size, num_classes, bottleneck_size=32):
        super(DenseAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),

            nn.Linear(64, bottleneck_size),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),

            nn.Linear(64, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),

            nn.Linear(128, input_size),
            nn.Sigmoid()  # Output in [0, 1] range
        )

        # Classifier (uses bottleneck features)
        self.classifier = nn.Sequential(
            nn.Linear(bottleneck_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x, return_bottleneck=False):
        # Encode
        bottleneck = self.encoder(x)

        # Decode
        reconstructed = self.decoder(bottleneck)

        # Classify
        classification = self.classifier(bottleneck)

        if return_bottleneck:
            return reconstructed, classification, bottleneck
        return reconstructed, classification

    def encode(self, x):
        """Get bottleneck representation"""
        return self.encoder(x)


# ==========================================================
# 🏋️ TRAINING FUNCTIONS
# ==========================================================

def train_epoch(model, data_generator, optimizer, device, alpha=0.5):
    """
    Train for one epoch with combined loss
    alpha: weight between reconstruction and classification loss
    """
    model.train()

    total_loss = 0
    total_recon_loss = 0
    total_class_loss = 0
    total_samples = 0

    reconstruction_criterion = nn.MSELoss()
    classification_criterion = nn.CrossEntropyLoss()

    for X_batch, y_batch in data_generator:
        # Convert to tensors
        X_tensor = torch.FloatTensor(X_batch).to(device)
        y_tensor = torch.LongTensor(y_batch).to(device)

        optimizer.zero_grad()

        # Forward pass
        reconstructed, classification = model(X_tensor)

        # Reconstruction loss (autoencoder)
        recon_loss = reconstruction_criterion(reconstructed, X_tensor)

        # Classification loss
        class_loss = classification_criterion(classification, y_tensor)

        # Combined loss
        loss = alpha * recon_loss + (1 - alpha) * class_loss

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(y_batch)
        total_recon_loss += recon_loss.item() * len(y_batch)
        total_class_loss += class_loss.item() * len(y_batch)
        total_samples += len(y_batch)

        # Free memory
        del X_tensor, y_tensor, reconstructed, classification
        torch.cuda.empty_cache()

    avg_loss = total_loss / total_samples
    avg_recon = total_recon_loss / total_samples
    avg_class = total_class_loss / total_samples

    return avg_loss, avg_recon, avg_class


def evaluate(model, data_generator, device):
    """Evaluate model"""
    model.eval()

    total_loss = 0
    total_recon_loss = 0
    total_class_loss = 0
    correct = 0
    total_samples = 0

    reconstruction_criterion = nn.MSELoss()
    classification_criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for X_batch, y_batch in data_generator:
            X_tensor = torch.FloatTensor(X_batch).to(device)
            y_tensor = torch.LongTensor(y_batch).to(device)

            reconstructed, classification = model(X_tensor)

            recon_loss = reconstruction_criterion(reconstructed, X_tensor)
            class_loss = classification_criterion(classification, y_tensor)
            loss = 0.5 * recon_loss + 0.5 * class_loss

            _, predicted = torch.max(classification, 1)
            correct += (predicted == y_tensor).sum().item()

            total_loss += loss.item() * len(y_batch)
            total_recon_loss += recon_loss.item() * len(y_batch)
            total_class_loss += class_loss.item() * len(y_batch)
            total_samples += len(y_batch)

            del X_tensor, y_tensor, reconstructed, classification
            torch.cuda.empty_cache()

    accuracy = correct / total_samples
    avg_loss = total_loss / total_samples
    avg_recon = total_recon_loss / total_samples
    avg_class = total_class_loss / total_samples

    return avg_loss, avg_recon, avg_class, accuracy


# ==========================================================
# 📂 DOWNLOAD & SPLIT DATASET
# ==========================================================

print("=" * 80)
print("📥 Downloading CIC-IoT-2023 Dataset from Kaggle...")
print("=" * 80)

dataset_dir = kagglehub.dataset_download("akashdogra/cic-iot-2023")
print(f"✅ Dataset downloaded to: {dataset_dir}")

csv_files = sorted([
    os.path.join(dataset_dir, f)
    for f in os.listdir(dataset_dir)
    if f.endswith(".csv")
])

print(f"📂 Found {len(csv_files)} CSV files.")

# 60-20-20 split
n_files = len(csv_files)
train_idx = int(n_files * 0.60)
val_idx = int(n_files * 0.80)

train_files = csv_files[:train_idx]
val_files = csv_files[train_idx:val_idx]
test_files = csv_files[val_idx:]

print(f"\n📊 Dataset Split:")
print(f"   Training:   {len(train_files)} files")
print(f"   Validation: {len(val_files)} files")
print(f"   Testing:    {len(test_files)} files")

# ==========================================================
# 🏷️ FIT LABEL ENCODER
# ==========================================================

print("\n" + "=" * 80)
print("🏷️  Fitting Label Encoder...")
print("=" * 80)

all_labels = []
max_batch = 5

for i in range(0, len(train_files), max_batch):
    batch_files = train_files[i:i+max_batch]
    print(f"Processing batch {i//max_batch + 1}/{(len(train_files)-1)//max_batch + 1}")

    for f in batch_files:
        _, y = load_and_clean(f)
        all_labels.extend(list(y.astype(str)))

    if i % (max_batch * 4) == 0:
        gc.collect()

label_encoder = LabelEncoder()
label_encoder.fit(all_labels)
del all_labels
gc.collect()

print(f"✅ LabelEncoder fitted with {len(label_encoder.classes_)} classes")

# ==========================================================
# 🏗️ FIT SCALER & PCA
# ==========================================================

print("\n" + "=" * 80)
print("🏗️  Fitting Scaler & PCA...")
print("=" * 80)

scaler = StandardScaler()

sample_X, _ = load_and_clean(train_files[0])
sample_X = encode_objects(sample_X)
n_features = sample_X.shape[1]
n_components = min(30, n_features)
del sample_X
gc.collect()

print(f"PCA will use {n_components} components (dataset has {n_features} features)")

pca = IncrementalPCA(n_components=n_components)

# Pass 1: Fit Scaler
print("Pass 1: Fitting Scaler...")
for i in range(0, len(train_files), max_batch):
    batch_files = train_files[i:i+max_batch]
    print(f"  Scaler batch {i//max_batch + 1}/{(len(train_files)-1)//max_batch + 1}")

    for f in batch_files:
        X, _ = load_and_clean(f)
        X = encode_objects(X)
        scaler.partial_fit(X)
        del X
        gc.collect()

print("✅ Scaler fitted")

# Pass 2: Fit PCA
print("\nPass 2: Fitting PCA...")
for i in range(0, len(train_files), max_batch):
    batch_files = train_files[i:i+max_batch]
    print(f"  PCA batch {i//max_batch + 1}/{(len(train_files)-1)//max_batch + 1}")

    for f in batch_files:
        X, _ = load_and_clean(f)
        X = encode_objects(X)
        X_scaled = scaler.transform(X)
        pca.partial_fit(X_scaled)
        del X, X_scaled
        gc.collect()

print(f"✅ PCA fitted with {pca.n_components_} components")
gc.collect()

# ==========================================================
# 🏗️ BUILD AUTOENCODER MODEL
# ==========================================================

print("\n" + "=" * 80)
print("🏗️  Building Autoencoder Model...")
print("=" * 80)

n_classes = len(label_encoder.classes_)
bottleneck_size = 32

model = DenseAutoencoder(
    input_size=n_components,
    num_classes=n_classes,
    bottleneck_size=bottleneck_size
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
encoder_params = sum(p.numel() for p in model.encoder.parameters())
decoder_params = sum(p.numel() for p in model.decoder.parameters())
classifier_params = sum(p.numel() for p in model.classifier.parameters())

print(f"\n📊 Model Architecture:")
print(f"   Input Size:      {n_components}")
print(f"   Bottleneck Size: {bottleneck_size}")
print(f"   Output Classes:  {n_classes}")
print(f"\n📊 Parameter Count:")
print(f"   Encoder:     {encoder_params:,}")
print(f"   Decoder:     {decoder_params:,}")
print(f"   Classifier:  {classifier_params:,}")
print(f"   Total:       {total_params:,}")

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ==========================================================
# 🎯 TRAINING
# ==========================================================

print("\n" + "=" * 80)
print("🎯 Starting Training...")
print("=" * 80)

epochs = 30
files_per_epoch = 3
alpha = 0.5  # Weight between reconstruction and classification

best_val_acc = 0
patience_counter = 0
patience = 5

training_history = {
    'train_loss': [],
    'train_recon': [],
    'train_class': [],
    'val_loss': [],
    'val_recon': [],
    'val_class': [],
    'val_acc': []
}

print(f"\nTraining Configuration:")
print(f"   Epochs: {epochs}")
print(f"   Files per epoch: {files_per_epoch}")
print(f"   Loss weight (alpha): {alpha}")
print(f"   Optimizer: Adam (lr=0.001)")

for epoch in range(epochs):
    print(f"\n{'='*80}")
    print(f"EPOCH {epoch+1}/{epochs}")
    print(f"{'='*80}")

    # Select training files
    start = (epoch * files_per_epoch) % len(train_files)
    selected_files = train_files[start:start + files_per_epoch]

    if len(selected_files) < files_per_epoch:
        selected_files += train_files[:files_per_epoch - len(selected_files)]

    print(f"Training on {len(selected_files)} files")

    # Train
    train_gen = process_files_generator(selected_files, scaler, pca, label_encoder, batch_size=files_per_epoch)
    train_loss, train_recon, train_class = train_epoch(model, train_gen, optimizer, device, alpha)

    # Validate
    val_gen = process_files_generator(val_files[:5], scaler, pca, label_encoder, batch_size=5)
    val_loss, val_recon, val_class, val_acc = evaluate(model, val_gen, device)

    # Store history
    training_history['train_loss'].append(train_loss)
    training_history['train_recon'].append(train_recon)
    training_history['train_class'].append(train_class)
    training_history['val_loss'].append(val_loss)
    training_history['val_recon'].append(val_recon)
    training_history['val_class'].append(val_class)
    training_history['val_acc'].append(val_acc)

    print(f"Train Loss: {train_loss:.4f} (Recon: {train_recon:.4f}, Class: {train_class:.4f})")
    print(f"Val Loss: {val_loss:.4f} (Recon: {val_recon:.4f}, Class: {val_class:.4f}) | Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'input_size': n_components,
            'num_classes': n_classes,
            'bottleneck_size': bottleneck_size
        }, 'autoencoder_model.pth')
        print(f"✅ Best model saved! Val Acc: {val_acc:.4f}")
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"\n⚠️  Early stopping triggered at epoch {epoch+1}")
        break

    gc.collect()
    torch.cuda.empty_cache()

print("\n✅ Training Complete!")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")

# Load best model
checkpoint = torch.load('autoencoder_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])

# ==========================================================
# 📈 FINAL EVALUATION ON TEST SET
# ==========================================================

print("\n" + "=" * 80)
print("📈 Final Evaluation on Test Set")
print("=" * 80)

model.eval()
y_true_all = []
y_pred_all = []

test_gen = process_files_generator(test_files, scaler, pca, label_encoder, batch_size=5)

with torch.no_grad():
    for batch_num, (X_test, y_test) in enumerate(test_gen):
        print(f"Test batch {batch_num + 1}/{(len(test_files)-1)//5 + 1}")

        X_tensor = torch.FloatTensor(X_test).to(device)
        _, classification = model(X_tensor)
        _, predicted = torch.max(classification, 1)

        y_true_all.extend(y_test)
        y_pred_all.extend(predicted.cpu().numpy())

        del X_tensor, classification
        torch.cuda.empty_cache()

y_true_all = np.array(y_true_all)
y_pred_all = np.array(y_pred_all)

# Calculate metrics
accuracy = accuracy_score(y_true_all, y_pred_all)
precision = precision_score(y_true_all, y_pred_all, average='weighted', zero_division=0)
recall = recall_score(y_true_all, y_pred_all, average='weighted', zero_division=0)
f1 = f1_score(y_true_all, y_pred_all, average='weighted', zero_division=0)

print(f"\n📊 Test Set Performance:")
print(f"   Accuracy:  {accuracy:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1-Score:  {f1:.4f}")

# ==========================================================
# 📊 CONFUSION MATRIX
# ==========================================================

print("\n" + "=" * 80)
print("📊 Generating Confusion Matrix...")
print("=" * 80)

cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(20, 16))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Count'})
plt.title('Autoencoder Confusion Matrix - IoT Intrusion Detection', fontsize=16, pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(rotation=0, fontsize=8)
plt.tight_layout()
plt.savefig('autoencoder_confusion_matrix.png', dpi=300, bbox_inches='tight')
print("✅ Confusion matrix saved as 'autoencoder_confusion_matrix.png'")

# ==========================================================
# 📈 TRAINING HISTORY PLOTS
# ==========================================================

print("\n" + "=" * 80)
print("📈 Generating Training History Plots...")
print("=" * 80)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Total Loss
axes[0, 0].plot(training_history['train_loss'], label='Train Loss', marker='o')
axes[0, 0].plot(training_history['val_loss'], label='Val Loss', marker='s')
axes[0, 0].set_title('Total Loss Over Epochs')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Plot 2: Reconstruction Loss
axes[0, 1].plot(training_history['train_recon'], label='Train Recon', marker='o')
axes[0, 1].plot(training_history['val_recon'], label='Val Recon', marker='s')
axes[0, 1].set_title('Reconstruction Loss Over Epochs')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('MSE Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Plot 3: Classification Loss
axes[1, 0].plot(training_history['train_class'], label='Train Class', marker='o')
axes[1, 0].plot(training_history['val_class'], label='Val Class', marker='s')
axes[1, 0].set_title('Classification Loss Over Epochs')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Cross-Entropy Loss')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Plot 4: Validation Accuracy
axes[1, 1].plot(training_history['val_acc'], label='Val Accuracy', marker='s', color='green')
axes[1, 1].set_title('Validation Accuracy Over Epochs')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
print("✅ Training history saved as 'training_history.png'")

# ==========================================================
# 💾 SAVE MODEL AND ARTIFACTS
# ==========================================================

print("\n" + "=" * 80)
print("💾 Saving Model and Artifacts")
print("=" * 80)

# Save preprocessing objects
preprocessing_objects = {
    'scaler': scaler,
    'pca': pca,
    'label_encoder': label_encoder
}

with open('preprocessing.pkl', 'wb') as f:
    pickle.dump(preprocessing_objects, f)
print("✅ Saved: preprocessing.pkl")

# Save metadata
metadata = {
    'n_classes': int(n_classes),
    'n_features': int(n_features),
    'n_components': int(n_components),
    'bottleneck_size': int(bottleneck_size),
    'total_params': int(total_params),
    'encoder_params': int(encoder_params),
    'decoder_params': int(decoder_params),
    'classifier_params': int(classifier_params),
    'test_accuracy': float(accuracy),
    'test_precision': float(precision),
    'test_recall': float(recall),
    'test_f1': float(f1),
    'best_val_acc': float(best_val_acc),
    'classes': label_encoder.classes_.tolist(),
    'training_history': {k: [float(v) for v in vals] for k, vals in training_history.items()}
}

with open('autoencoder_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=4)
print("✅ Saved: autoencoder_metadata.json")

# Create summary
with open('autoencoder_summary.txt', 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("AUTOENCODER MODEL SUMMARY\n")
    f.write("=" * 80 + "\n\n")

    f.write("MODEL ARCHITECTURE:\n")
    f.write(f"  Input Size:      {n_components}\n")
    f.write(f"  Bottleneck Size: {bottleneck_size}\n")
    f.write(f"  Output Classes:  {n_classes}\n\n")

    f.write("PARAMETER COUNT:\n")
    f.write(f"  Encoder:     {encoder_params:,}\n")
    f.write(f"  Decoder:     {decoder_params:,}\n")
    f.write(f"  Classifier:  {classifier_params:,}\n")
    f.write(f"  Total:       {total_params:,}\n\n")

    f.write("TEST SET PERFORMANCE:\n")
    f.write(f"  Accuracy:  {accuracy:.4f}\n")
    f.write(f"  Precision: {precision:.4f}\n")
    f.write(f"  Recall:    {recall:.4f}\n")
    f.write(f"  F1-Score:  {f1:.4f}\n\n")

    f.write("FILES GENERATED:\n")
    f.write("  - autoencoder_model.pth (Model checkpoint)\n")
    f.write("  - preprocessing.pkl (Scaler, PCA, Label Encoder)\n")
    f.write("  - autoencoder_metadata.json (Model specifications)\n")
    f.write("  - autoencoder_confusion_matrix.png (Confusion matrix)\n")
    f.write("  - training_history.png (Training curves)\n")

print("✅ Saved: autoencoder_summary.txt")

# List all files
print("\n" + "=" * 80)
print("📁 Generated Files:")
print("=" * 80)

files_to_check = [
    'autoencoder_model.pth',
    'preprocessing.pkl',
    'autoencoder_metadata.json',
    'autoencoder_summary.txt',
    'autoencoder_confusion_matrix.png',
    'training_history.png'
]

total_size = 0
for filename in files_to_check:
    if os.path.exists(filename):
        size = os.path.getsize(filename)
        total_size += size
        size_mb = size / (1024 * 1024)
        print(f"✅ {filename:<45} {size_mb:>10.2f} MB")

print("=" * 80)
print(f"📊 Total Size: {total_size / (1024 * 1024):.2f} MB")

print("\n" + "=" * 80)
print("🎉 AUTOENCODER TRAINING COMPLETE!")
print("=" * 80)
print(f"\nFinal Test Accuracy: {accuracy:.4f}")
print(f"Model Parameters: {total_params:,}")
print("\nAll files have been saved and are ready for download.")
print("=" * 80)

🎮 GPU Configuration
✅ GPU detected: Tesla T4
✅ CUDA Version: 12.6
✅ GPU Memory: 15.83 GB
✅ Number of GPUs: 1
✅ cuDNN autotuner enabled

📥 Downloading CIC-IoT-2023 Dataset from Kaggle...


100%|██████████| 2.77G/2.77G [00:28<00:00, 106MB/s]

Extracting files...


✅ Dataset downloaded to: /root/.cache/kagglehub/datasets/akashdogra/cic-iot-2023/versions/1
📂 Found 169 CSV files.

📊 Dataset Split:
   Training:   101 files
   Validation: 34 files
   Testing:    34 files

🏷️  Fitting Label Encoder...
Processing batch 1/21
Processing batch 2/21
Processing batch 3/21
Processing batch 4/21
Processing batch 5/21
Processing batch 6/21
Processing batch 7/21
Processing batch 8/21
Processing batch 9/21
Processing batch 10/21
Processing batch 11/21
Processing batch 12/21
Processing batch 13/21
Processing batch 14/21
Processing batch 15/21
Processing batch 16/21
Processing batch 17/21
Processing batch 18/21
Processing batch 19/21
Processing batch 20/21
Processing batch 21/21
✅ LabelEncoder fitted with 34 classes

🏗️  Fitting Scaler & PCA...
PCA will use 30 components (dataset has 46 features)
Pass 1: Fitting Scaler...
  Scaler batch 1/21
  Scaler batch 2/21
  Scaler batch 3/21
  Scaler batch 4/21
  Scaler batch 5/21
  Scaler batch 6/21
  Scaler batch 7/21
  Sc